In [1]:
import requests
import pandas as pd
import json
import ipywidgets as widgets
from datetime import date

In [2]:
# read historical csv and read to data frame
csv_name = 'historical_DOGE-USD.csv'
prices_df = pd.read_csv(csv_name)
prices_df.head()

,Date,High
0,2021-05-05,0.677000
1,2014-09-17,0.000299
2,2014-09-18,0.000325
3,2014-09-19,0.000307
4,2014-09-20,0.000310


In [3]:
# get current price of doge and add overwrite todays entry
json_response = requests.get('https://sochain.com//api/v2/get_price/DOGE/USD').json()

i = 0
current_price = -1
if json_response['status'] == 'success':
  while current_price <= 0:
    current_price = float(json_response['data']['prices'][i]['price'])
    i += 1

print(current_price)

0.677


In [4]:
# add today's price to the csv for next run. keep the highest price when duplicate dates
df_today = {'Date': date.today(), 'High': current_price}
prices_df = prices_df.append(df_today, ignore_index = True)
prices_df = prices_df.sort_values(['Date', 'High'] , ascending=True).drop_duplicates(['Date'] , keep='last').reset_index(drop=True)
prices_df.set_index(['Date']).to_csv(csv_name)

In [5]:
# accept date from user
calendar = widgets.DatePicker(
    description='Select Date',
    disabled=False
)
calendar

DatePicker(value=None, description='Select Date')

In [6]:
# accept money invested on date from user
investment_input = widgets.BoundedFloatText(
    value=100.00,
    min=0,
    max=1_000_000_000_000,
    description='Investment $',
    disabled=False
)
investment_input

BoundedFloatText(value=100.0, description='Investment $', max=1000000000000.0)

In [7]:
# read calendar widget
buy_date = str(calendar.value)

# get price on day in calendar widget
price_on_date = prices_df[prices_df['Date']==buy_date]['High'].values[0]

# read initial investment widget
initial_investment = investment_input.value

# calculate number of coins bought with initial investment
coins_bought = initial_investment / price_on_date

# calculate current value
current_value = coins_bought * current_price

# calculate percent returns
percent_change = (current_value - initial_investment) / initial_investment * 100

# display output
print(f"""initial_investment = ${round(initial_investment,2)}, 
buy_date = {buy_date},
buy_date_price = ${round(price_on_date,2)},
coins_bought = {round(coins_bought,2)}, 
current_price = ${round(current_price,2)},
current_value = ${round(current_value,2)}, 
percent_change = {round(percent_change,2)}%""")


initial_investment = $100.0, 
buy_date = 2021-04-26,
buy_date_price = $0.28,
coins_bought = 356.57, 
current_price = $0.68,
current_value = $241.4, 
percent_change = 141.4%


In [ ]:
# TODO: Make into Viola file

In [ ]:
# TODO: Make WHAT-IF simulations for what-if buy price or what-if sell price